<font color=red>注：此处是文档第42页</font>

## 数据并行处理-GPU

在这个教程中，我们将学习如何用 DataParallel 来使用多 GPU。 通过 PyTorch 使用多个GPU非常简单。你可以将模型放在一个 GPU：
```python
device = torch.device('cuda: 0')
model.to(device)
```
然后可以复制所有的张量到GPU：
```python
mytensor = my_tensor.to(devive)
```
请注意，只是调用`my_tensor.to(device)`返回一个`my_tensor`新的复制在GPU上，而不是重写`my_tensor`。你需要分配给他一个新的张量并且在GPU上使用这个张量。

在多GPU中执行前馈，后馈操作是非常自然的。尽管如此，PyTorch默认只会使用一个GPU。通过使用`DataParallel`让你的模型并行运行，你可以很容易的在多GPU上运行你的操作。 

```python
model = nn.DataParallel(model)
```

这是整个教程的核心，我们接下来将会详细讲解。 引用和参数

引入 PyTorch 模块和定义参数 

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

#### 参数

In [7]:
input_size = 5
output_size = 2

batch_size = 30
data_size = 100

#### 设备

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
device

device(type='cuda', index=0)

#### 实验（玩具）数据

生成一个玩具数据。你只需要实现 getitem.

In [10]:
class RandomDataset(Dataset):
    
    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)
        
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return self.len
    
rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size), 
                         batch_size=batch_size, shuffle=True)

#### 简单模型

为了做一个小demo，我们的模型只是获得一个输入，执行一个线性操作，然后给一个输出。尽管如此，你可以使用DataParallel在任何模型(CNN, RNN, Capsule Net 等等.)

我们放置了一个输出声明在模型中来检测输出和输入张量的大小。请注意在 batch rank 0 中的输出。 

In [12]:
class Model(nn.Module):
    
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        
    def forward(self, input):
        output = self.fc(input)
        print("\tIn Model: input size", input.size(), 
             "output size:", output.size())
        return output

#### 创建模型并且数据并行处理

这是整个教程的核心。首先我们需要一个模型的实例，然后验证我们是否有多个 GPU。如果我们有多个GPU，我们可以用 nn.DataParallel 来包裹我们的模型。然后我们使用 model.to(device) 把模型放到多 GPU 中。 

In [16]:
model = Model(input_size, output_size)
if torch.cuda.device_count() >= 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
model.to(device)

Let's use 1 GPUs!


DataParallel(
  (module): Model(
    (fc): Linear(in_features=5, out_features=2, bias=True)
  )
)

In [18]:
for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("Outside: input size", input.size(),
          "output_size", output.size())

	In Model: input size torch.Size([30, 5]) output size: torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size: torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size: torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([10, 5]) output size: torch.Size([10, 2])
Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])


### 总结
数据并行自动拆分了你的数据并且将任务单发送到多个 GPU 上。当每一个模型都完成自己的任务之后，DataParallel 收集并且合并这些结果，然后再返回给你。
更多信息，请访问: [https://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html](https://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html)